<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/carseats.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 평가를 진행합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [76]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [78]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Carseats.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**데이터 설명**

- **Sales: 각 지역 판매량(단위: 1,000개)**
- CompPrice: 경쟁사 가격 (단위: 달러)
- Income: 지역 평균 소득 (단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산 (단위: 1,000달러)
- Population: 지역 인구 수 (단위: 1,000명)
- Price: 자사 지역별 판매 가격 (단위: 달러)
- ShelveLoc: 진열 상태
- Age: 지역 인구의 평균 연령
- Education: 각 지역 교육 수준
- Urban: 도심 지역 여부 (Yes,No)
- US: 매장이 미국에 있는지 여부 (Yes, No)

In [7]:
# 데이터 살펴보기
data.head()



,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [11]:
# 기술통계 확인

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


 변수 관련 정보 확인

Sales가 타깃

ShelveLo,Education,Urban,US 가 데이터타입이 object이므로 가변수화(문자->숫자)를 진행



# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [80]:
# x, y 분리
target='Sales'
x=data.drop(columns=target)
y=data.loc[:,target]

**2) 가변수화**

In [83]:
# 가변수화
dummies=['ShelveLoc','Education','Urban','US']

x=pd.get_dummies(x,columns=dummies,drop_first=True,dtype=int)

x

,CompPrice,Income,Advertising,Population,Price,Age,ShelveLoc_Good,ShelveLoc_Medium,Education_11,Education_12,Education_13,Education_14,Education_15,Education_16,Education_17,Education_18,Urban_Yes,US_Yes
0,138,73,11,276,120,42,0,0,0,0,0,0,0,0,1,0,1,1
1,111,48,16,260,83,65,1,0,0,0,0,0,0,0,0,0,1,1
2,113,35,10,269,80,59,0,1,0,1,0,0,0,0,0,0,1,1
3,117,100,4,466,97,55,0,1,0,0,0,1,0,0,0,0,1,1
4,141,64,3,340,128,38,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,33,1,0,0,0,0,1,0,0,0,0,1,1
396,139,23,3,37,120,55,0,1,1,0,0,0,0,0,0,0,0,1
397,162,26,12,368,159,40,0,1,0,0,0,0,0,0,0,1,1,1
398,100,79,7,284,95,50,0,0,0,1,0,0,0,0,0,0,1,1


**3) 학습용, 평가용 데이터 분리**

In [85]:
# 학습용, 평가용 데이터 분리
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)


**4) 정규화**

In [30]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(x_train)
x_train_s=scaler.transform(x_train)
x_test_s=scaler.transform(x_test)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 다양한 알고리즘으로 모델을 만들고 성능을 확입니다.
- 성능 평가는 mean_absolute_error, r2_score를 사용합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 사용할 알고리즘을 모두 불러옵니다.

In [87]:
# 라이브러리 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import *

**1) Linear Regression**

- Linear Regression 알고리즘으로 모델링하고 성능을 확인합니다.

In [67]:
# 선언하기
model=LinearRegression()

# 학습하기
model.fit(x_train,y_train)

# 예측하기
y_pred=model.predict(x_test_s)

# 평가하기
print('MAE:',mean_absolute_error(y_test,y_pred))
print('R2:',r2_score(y_test,y_pred))

MAE: 1.8967718274736465
R2: 0.24825461847683272


**2) KNN**

- KNN 알고리즘으로 모델링하고 성능을 확인합니다.

In [69]:
# 선언하기

model=KNeighborsClassifier(n_neighbors=5)
# 학습하기
model.fit(x_train_s,y_train)

# 예측하기
y_pred=model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

**3) Decision Tree**

- Decision Tree 알고리즘으로 모델링하고 성능을 확인합니다.

In [71]:
# 선언하기
model = DecisionTreeClassifier(max_depth=5, random_state=1)

# 학습하기
model.fit(x_train_s, y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

**4) Random Forest**

- Random Forest 알고리즘으로 모델링하고 성능을 확인합니다.

In [60]:
# 선언하기
model=RandomForestClassifier(max_depth=5)

# 학습하기
model.fit(x_train,y_train)

# 예측하기
y_pred=model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

**5) XGBoost**

- XGBoost 알고리즘으로 모델링하고 성능을 확인합니다.

In [65]:
# 선언하기
model=XGBClassifier(max_depth=5)

# 학습하기
model.fit(x_train,y_train)

# 예측하기
y_pred=model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 

**6) LightGBM**

- LightGBM 알고리즘으로 모델링하고 성능을 확인합니다.

In [ ]:
# 선언하기


# 학습하기


# 예측하기


# 평가하기

